In [1]:
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from sklearn.metrics import r2_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold, cross_val_predict, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from scipy import stats
import statsmodels.api as sm

In [3]:
import pickle as pkl

In [4]:
religion_feats = ['Catholic Church', 'Buddhism', 'Atheism']
interest_feats = ['Gambling', 'Casino', 'Cooking', 'Restaurants',  'Fast food', 'Fitness and wellness',  'LGBT community','Homosexuality', 'Same-sex marriage']

In [5]:
q_list = [25, 50, 75]

In [6]:
# Number of trees in random forest
n_estimators_list = [10, 50, 100, 200, 300]
# Maximum number of levels in tree
max_depth_list = [2, 5, 20, 50, 100]
# max_depth_list.append(None)
# Minimum number of samples required to split a node
min_samples_split_list = [2]
# Minimum number of samples required at each leaf node
min_samples_leaf_list = [1]

# ALL AGES

In [7]:
dfs = {}
ys = {}

In [8]:
df_city_zips = pd.read_csv('data/atlanta/median_income_city.csv')
df = pd.read_csv('data/atlanta/atlanta_all_ages_normalized_average_query_withnans.csv').drop(labels=['Unnamed: 0'], axis=1).fillna(0.0)
df = df[df.zips.isin(df_city_zips.name.iloc[1:].astype(int))].reset_index(drop=True)

In [9]:
#ATLANTA
df_city_zips = pd.read_csv('data/atlanta/median_income_city.csv')
df = pd.read_csv('data/atlanta/atlanta_all_ages_normalized_average_query_withnans.csv').drop(labels=['Unnamed: 0'], axis=1).fillna(0.0)
df = df[df.zips.isin(df_city_zips.name.iloc[1:].astype(int))].reset_index(drop=True)
    
zips = (df.loc[:,'Users':'Windows']!=0).sum(axis=1)>0
df = df[zips]

features = df.loc[:,'Users':].columns[(df.loc[:,'Users':]!=0).sum(axis=0) > 0]
df = df[features]

for q in q_list:
    df['incomeq%s'%q] = (df.income > np.percentile(df.income, q)).astype(int)

dfs['atlanta'] = df.loc[:,~df.columns.isin(interest_feats+religion_feats)].loc[:, 'Males':'Windows']
ys['atlanta'] = df[['income']+['incomeq%s'%q for q in q_list]]

In [10]:
#BOGOTA
df = pd.read_csv('data/bogota/bogota_all_ages_normalized_average_query_withnans.csv').drop(labels=['Unnamed: 0'], axis=1)

df['income'] = df.median_income
    
zips = (df.loc[:,'Users':'Windows']!=0).sum(axis=1)>0
df = df[zips]

features = df.loc[:,'Users':].columns[(df.loc[:,'Users':]!=0).sum(axis=0) > 0]
df = df[features]

rs = np.random.RandomState(seed=0)
np_income = df.income + rs.rand(df.income.shape[0])*1e-5
for q in q_list:
    df['incomeq%s'%q] = (np_income > np.percentile(np_income, q)).astype(int)

dfs['bogota'] = df.loc[:,~df.columns.isin(interest_feats+religion_feats)].loc[:, 'Males':'Windows']
ys['bogota'] = df[['income']+['incomeq%s'%q for q in q_list]]

In [11]:
#SANTIAGO
df = pd.read_csv('data/santiago/santiago_all_ages_normalized_average_query_withnans.csv').drop(labels=['Unnamed: 0'], axis=1).dropna()
df = df[(df.comuna_id < 13200) & (df.comuna_id!=13115)].reset_index(drop=True)

df['income'] = df.median_income_comuna
    
zips = (df.loc[:,'Users':'Windows']!=0).sum(axis=1)>0
df = df[zips]

features = df.loc[:,'Users':].columns[(df.loc[:,'Users':]!=0).sum(axis=0) > 0]
df = df[features]

rs = np.random.RandomState(seed=0)
np_income = df.income + rs.rand(df.income.shape[0])
for q in q_list:
    df['incomeq%s'%q] = (np_income > np.percentile(np_income, q)).astype(int)

dfs['santiago'] = df.loc[:,~df.columns.isin(interest_feats+religion_feats)].loc[:, 'Males':'Windows']
ys['santiago'] = df[['income']+['incomeq%s'%q for q in q_list]]

In [12]:
#CASABLANCA
df = pd.read_csv('data/casablanca/casablanca_all_ages_normalized_average_query_withnans.csv').drop(labels=['Unnamed: 0'], axis=1)

df['income'] = df.Taux_de_pauvretmultidimensionnelle2014
    
zips = (df.loc[:,'Users':'Windows']!=0).sum(axis=1)>0
df = df[zips]

features = df.loc[:,'Users':].columns[(df.loc[:,'Users':]!=0).sum(axis=0) > 0]
df = df[features]

rs = np.random.RandomState(seed=0)
np_income = df.income + rs.rand(df.income.shape[0])*1e-5
for q in q_list:
    df['incomeq%s'%np.abs(q - 100)] = (np_income < np.percentile(np_income, q)).astype(int)

dfs['casablanca'] = df.loc[:,~df.columns.isin(interest_feats+religion_feats)].loc[:, 'Males':'Windows']
ys['casablanca'] = df[['income']+['incomeq%s'%q for q in q_list]]

In [ ]:
city_train = 'santiago'
city_test = 'bogota'

best_qs = []
df_train = dfs[city_train]
X_train = df_train.values

SEED = 42

for q_train in q_list:

    rs = np.random.RandomState(SEED)

    y_train = ys[city_train]['incomeq%s'%q_train].values
    y_true = ys[city_train]['income'].values

    for n_estimators in n_estimators_list:
        for max_depth in max_depth_list:
            for min_samples_split in min_samples_split_list:
                for min_samples_leaf in min_samples_leaf_list:

                    params = [n_estimators, max_depth, min_samples_split, min_samples_leaf]

                    aucs = []

                    for seed in range(30):

                        rfc = RandomForestClassifier(n_estimators=n_estimators,
                                                      max_depth=max_depth, 
                                                      min_samples_split=min_samples_split,
                                                      min_samples_leaf=min_samples_leaf, 
                                                      random_state=rs, n_jobs=-1)
                        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=rs)
                        cv_folds = list(cv.split(X_train,y_train))
                        model = make_pipeline(StandardScaler(), rfc)

                        y_probs = cross_val_predict(model, X_train, y_train, cv=cv_folds, method='predict_proba')

                        auc = roc_auc_score(y_train, y_probs[:, 1])
                        aucs.append(auc)
                    best_qs.append((np.mean(aucs), np.std(aucs), params, q_train))

best_qtrains = sorted([b for b in best_qs if ~np.isnan(b[0])], key= lambda x: x[0], reverse=True)
best_qtrains = [[b for b in best_qtrains if b[-1]==q][0] for q in q_list]

print(city_train)
for b in best_qtrains:
    print(b[2][:2], [b[-1]], 'roc-auc: %.3f %s %.3f' % (b[0], u"\u00B1", b[1]))
    
if city_train != city_test:
    
    rs = np.random.RandomState(SEED)

    df_test = dfs[city_test]
    feats = list(set(df_train.columns) & set(df_test.columns))
    X_train = df_train.loc[:, df_train.columns.isin(feats)].values
    X_test = df_test.loc[:, df_test.columns.isin(feats)].values

    shuff_idx = rs.permutation(X_train.shape[0])

    scalar = StandardScaler()
    scalar.fit(X_train)
    X_train = scalar.transform(X_train)
    X_test = scalar.transform(X_test)

    print(city_train, city_test)

    for best_qtrain in best_qtrains:

        q_train = best_qtrain[-1]
        q_test = q_train
        y_train = ys[city_train]['incomeq%d'%q_train].values

        y_test = ys[city_test]['incomeq%s'%q_test].values
        y_true = ys[city_test]['income'].values

        aucs = []

        for seed in range(30):

            rfc = RandomForestClassifier(n_estimators=best_qtrain[2][0],
                                        max_depth=best_qtrain[2][1], random_state=seed, n_jobs=-1)
            rfc.fit(X_train[shuff_idx], y_train[shuff_idx])

            y_pred = rfc.predict(X_test)
            y_probs = rfc.predict_proba(X_test)

            auc = roc_auc_score(y_test, y_probs[:, 1])
            aucs.append(auc)

        print(best_qtrain[2][:2], [q_train, q_test], 'roc-auc: %.3f %s %.3f' % (np.mean(aucs), u"\u00B1", np.std(aucs)))